# 使用RNN实现情感分类


## 概述

情感分类是自然语言处理中的经典任务，是典型的分类问题。本节使用MindSpore实现一个基于RNN网络的情感分类模型，实现如下的效果：

```text
[> : input, = : target, < : output]

> This film is terrible
= Negtive
< Negtive

> This film is great
= Positive
< Positive
```

## 数据准备

本节使用情感分类的经典数据集[IMDB影评数据集](https://ai.stanford.edu/~amaas/data/sentiment/)，数据集包含Positive和Negative两类，下面为其样例：

| Review  | Label  |
|:---|:---:|
| "Quitting" may be as much about exiting a pre-ordained identity as about drug withdrawal. As a rural guy coming to Beijing, class and success must have struck this young artist face on as an appeal to separate from his roots and far surpass his peasant parents' acting success. Troubles arise, however, when the new man is too new, when it demands too big a departure from family, history, nature, and personal identity. The ensuing splits, and confusion between the imaginary and the real and the dissonance between the ordinary and the heroic are the stuff of a gut check on the one hand or a complete escape from self on the other.  |  Negative |  
| This movie is amazing because the fact that the real people portray themselves and their real life experience and do such a good job it's like they're almost living the past over again. Jia Hongsheng plays himself an actor who quit everything except music and drugs struggling with depression and searching for the meaning of life while being angry at everyone especially the people who care for him most.  | Positive  |

此外，需要使用预训练词向量对自然语言单词进行编码，以获取文本的语义特征，本节选取[Glove](https://nlp.stanford.edu/projects/glove/)词向量作为Embedding。

### 数据下载模块

为了方便数据集和预训练词向量的下载，首先设计数据下载模块，实现可视化下载流程，并保存至指定路径。数据下载模块使用`requests`库进行http请求，并通过`tqdm`库对下载百分比进行可视化。此外针对下载安全性，使用IO的方式下载临时文件，而后保存至指定的路径并返回。

In [1]:
import os
import logging
import shutil
import requests
import tempfile
from tqdm import tqdm
from typing import IO
from pathlib import Path

# 指定保存路径为 `home_path/.mindspore_examples`
cache_dir = Path.home() / '.mindspore_examples'

def http_get(url: str, temp_file:IO):
    """使用requests库下载数据，并使用tqdm库进行流程可视化"""
    req = requests.get(url, stream=True)
    content_length = req.headers.get('Content-Length')
    total = int(content_length) if content_length is not None else None
    progress = tqdm(unit='B', total=total)
    for chunk in req.iter_content(chunk_size=1024):
        if chunk:
            progress.update(len(chunk))
            temp_file.write(chunk)
    progress.close()

def download(file_name:str, url: str):
    """下载数据并存为指定名称"""
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    cache_path = os.path.join(cache_dir, file_name)
    cache_exist = os.path.exists(cache_path)
    if not cache_exist:
        with tempfile.NamedTemporaryFile() as temp_file:
            http_get(url, temp_file)
            temp_file.flush()
            temp_file.seek(0)
            logging.info(f"copying {temp_file.name} to cache at {cache_path}")
            with open(cache_path, 'wb') as cache_file:
                shutil.copyfileobj(temp_file, cache_file)
    return cache_path

完成数据下载模块后，下载IMDB数据集进行测试(此处使用华为云的镜像下线用于提升下载速度)。下载过程及保存的路径如下：

In [2]:
imdb_path = download('aclImdb_v1.tar.gz', 'https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/aclImdb_v1.tar.gz')
imdb_path

'/root/.mindspore_examples/aclImdb_v1.tar.gz'

### 加载IMDB数据集

下载好的IMDB数据集为`tar.gz`文件，我们使用Python的`tarfile`库对其进行读取，并将所有数据和标签分别进行存放。原始的IMDB数据集解压目录如下：

```text
    ├── aclImdb
    │   ├── imdbEr.txt
    │   ├── imdb.vocab
    │   ├── README
    │   ├── test
    │   └── train 
    │         ├── neg
    │         ├── pos
    ...
```

数据集已分割为train和test两部分，且每部分包含neg和pos两个分类的文件夹，因此需分别train和test进行读取并处理数据和标签。

In [3]:
import re
import six
import string
import tarfile

class IMDBData():
    """IMDB数据集加载器
    
    加载IMDB数据集并处理为一个Python迭代对象。
    
    """
    label_map = {
        "pos": 1,
        "neg": 0
    }
    def __init__(self, path, mode="train"):
        self.mode = mode
        self.path = path
        self.docs, self.labels = [], []

        self._load("pos")
        self._load("neg")

    def _load(self, label):
        pattern = re.compile(r"aclImdb/{}/{}/.*\.txt$".format(self.mode, label))
        # 将数据加载至内存
        with tarfile.open(self.path) as tarf:
            tf = tarf.next()
            while tf is not None:
                if bool(pattern.match(tf.name)):
                    # 对文本进行分词、去除标点和特殊字符、小写处理
                    self.docs.append(str(tarf.extractfile(tf).read().rstrip(six.b("\n\r"))
                        .translate(None, six.b(string.punctuation)).lower(
                        )).split())
                    self.labels.append([self.label_map[label]])
                tf = tarf.next()               

    def __getitem__(self, idx):
        return self.docs[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.docs)

完成IMDB数据加载器后，加载训练数据集进行测试，输出数据集数量：

In [4]:
imdb_train = IMDBData(imdb_path, 'train')
len(imdb_train)

25000

将IMDB数据集加载至内存并构造为迭代对象后，可以使用`mindspore.dataset`提供的`Generatordataset`接口进行加载进行下一步的数据处理，下面封装一个函数将train和test分别使用`Generatordataset`进行加载，并指定数据集中文本和标签的`column_name`分别为`text`和`label`:

In [5]:
import mindspore.dataset as dataset

def load_imdb(imdb_path):
    imdb_train = dataset.GeneratorDataset(IMDBData(imdb_path, "train"), column_names=["text", "label"])
    imdb_test = dataset.GeneratorDataset(IMDBData(imdb_path, "test"), column_names=["text", "label"])
    return imdb_train, imdb_test

加载IMDB数据集，可以看到imdb_train是一个GeneratorDataset对象

In [6]:
imdb_train, imdb_test = load_imdb(imdb_path)
imdb_train

### 加载预训练词向量

预训练词向量是对输入单词的数值化表示，通过`nn.Embedding`层，采用查表的方式，输入单词对应词表中的index，获得对应的表达向量。
因此进行模型构造前，需要将Embedding层所需的词向量和词表进行构造。这里我们使用Glove(Global Vectors for Word Representation)这种经典的预训练词向量，
其数据格式如下：

| Word |  Vector |  
|:---|:---:|
| the | 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 ...|  
| , | 0.013441 0.23682 -0.16899 0.40951 0.63812 0.47709 -0.42852 -0.55641 -0.364 ... | 

我们直接使用第一列的单词作为词表，使用`dataset.text.Vocab`将其按顺序加载；同时读取每一行的Vector并转为`numpy.array`，用于`nn.Embedding`加载权重使用。具体实现如下：

In [7]:
import zipfile
import numpy as np

def load_glove(glove_path):
    glove_100d_path = os.path.join(cache_dir, 'glove.6B.100d.txt')
    if not os.path.exists(glove_100d_path):
        glove_zip = zipfile.ZipFile(glove_path)
        glove_zip.extractall(cache_dir)

    embeddings = []
    tokens = []
    with open(glove_100d_path, encoding='utf-8') as gf:
        for glove in gf:
            word, embedding = glove.split(maxsplit=1)
            tokens.append(word)
            embeddings.append(np.fromstring(embedding, dtype=np.float32, sep=' '))
    # add embeddings for <unk> and <pad>
    embeddings.append(np.random.rand(100))
    embeddings.append(np.zeros((100,), np.float32))
    
    vocab = dataset.text.Vocab.from_list(tokens, special_tokens=["<unk>", "<pad>"], special_first=False)
    embeddings = np.array(embeddings).astype(np.float32)
    return vocab, embeddings

由于数据集中可能存在词表没有覆盖的单词，因此需要加入`<unk>`标记符；同时由于输入长度的不一致，在打包为一个batch时需要将短的文本进行填充，因此需要加入`<pad>`标记符。完成后的词表长度为原词表长度+2。

下面下载Glove词向量，并加载生成词表和词向量权重矩阵。

In [25]:
glove_path = download('glove.6B.zip', 'https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/glove.6B.zip')
vocab, embeddings = load_glove(glove_path)
len(vocab.vocab())

400002

使用词表将`the`转换为index id，并查询词向量矩阵对应的词向量：

In [9]:
idx = vocab.tokens_to_ids('the')
embedding = embeddings[idx]
idx, embedding

(0,
 array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
        -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
         0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
        -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
         0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
        -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
         0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
         0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
        -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
        -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
        -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
        -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
        -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
        -1.2526  ,  0.071624,  0.7

## 数据集预处理

通过加载器加载的IMDB数据集进行了分词处理，但不满足构造训练数据的需要，因此要对其进行额外的预处理。其中包含的预处理如下：

- 通过Vocab将所有的Token处理为index id。
- 将文本序列统一长度，不足的使用`<pad>`补齐，超出的进行截断

这里我们使用`mindspore.dataset`中提供的接口进行预处理操作。这里使用到的接口均为MindSpore的高性能数据引擎设计，每个接口对应操作视作数据流水线的一部分，详情请参考[MindSpore数据引擎]。(https://www.mindspore.cn/docs/programming_guide/zh-CN/r1.6/design/data_engine.html)
首先针对token到index id的查表操作，使用`text.Lookup`接口，将前文构造的词表加载，并指定`unknown_token`。其次为文本序列统一长度操作，使用`PadEnd`接口，此接口定义最大长度和补齐值(`pad_value`)，这里我们取最大长度为500，填充值对应词表中`<pad>`的index id.

> 除了对数据集中`text`进行预处理外，由于后续模型训练的需要，要将`label`数据转为float32格式

In [10]:
import mindspore

lookup_op = dataset.text.Lookup(vocab, unknown_token='<unk>')
pad_op = dataset.transforms.c_transforms.PadEnd([500], pad_value=vocab.tokens_to_ids('<pad>'))
type_cast_op = dataset.transforms.c_transforms.TypeCast(mindspore.float32)

完成预处理操作后，需将其加入到数据集处理流水线中，使用`map`接口对指定的column添加操作。

In [11]:
imdb_train = imdb_train.map(operations=[lookup_op, pad_op],  input_columns=['text'])
imdb_train = imdb_train.map(operations=[type_cast_op],  input_columns=['label'])

imdb_test = imdb_test.map(operations=[lookup_op, pad_op],  input_columns=['text'])
imdb_test = imdb_test.map(operations=[type_cast_op],  input_columns=['label'])

由于IMDB数据集本身不包含验证集，我们手动将其分割为训练和验证两部分，比例取0.7, 0.3。

In [12]:
imdb_train, imdb_valid = imdb_train.split([0.7, 0.3])

[WARNING] ME(19512:140655497611072,MainProcess):2022-02-24-03:01:51.925.6 [mindspore/dataset/engine/datasets.py:1064] Dataset is shuffled before split.


最后指定数据集的batch大小，通过`batch`接口指定，并设置是否丢弃无法被batch size整除的剩余数据。

> 调用数据集的`map`,`split`,`batch`均为数据集处理流水线增加对应操作，返回值为新的Dataset类型。现在仅定义流水线操作，在执行时开始执行数据处理流水线，获取最终处理好的数据并送入模型进行训练。

In [13]:
imdb_train = imdb_train.batch(64, drop_remainder=True)
imdb_valid = imdb_valid.batch(64, drop_remainder=True)

## 模型构建

完成数据集的处理后，我们设计用于情感分类的模型结构。首先需要将输入文本(即序列化后的index id列表)通过查表转为向量化表示，此时需要使用`nn.Embedding`层加载Glove词向量；然后使用RNN循环神经网络做特征提取；最后将RNN连接至一个全连接层，即`nn.Dense`，将特征转化为与分类数量相同的size，用于后续进行模型优化训练。整体模型结构如下：

```text
nn.Embedding -> nn.RNN -> nn.Dense
```

这里我们使用能够一定程度规避RNN梯度消失问题的变种LSTM(Long short term memory)做特征提取层。下面对模型进行详解：

### Embedding

Embedding层又可称为EmbeddingLookup层，其作用是使用index id对权重矩阵对应id的向量进行查找，当输入为一个由index id组成的序列时，则查找并返回一个相同长度的矩阵，例如：

```text
embedding = nn.Embedding(1000, 100) # 词表大小(index的取值范围)为1000，表示向量的size为100
input shape: (1, 16)                # 序列长度为16
output shape：(1, 16, 100)
```

这里我们使用前文处理好的Glove词向量矩阵，设置`nn.Embedding`的`embedding_table`为预训练词向量矩阵。对应的`vocab_size`为词表大小400002，`embedding_size`为选用的`glove.6B.100d`向量大小，即100。

#### RNN(循环神经网络)

$$(h_t, c_t) = \text{LSTM}(x_{0:t}, h_0, c_0)$$


![](assets/sentiment2.png)

##### Bidirectional RNN(双向RNN)

![](assets/sentiment3.png)

##### Multi-Layer RNN(多层RNN)

![](assets/sentiment4.png)

### Dense

> 在Dense层后进行了`sigmoid`运算，将预测值归一至`[0,1]`区间，用于后续和`BCELoss`(BinaryCrossEntropyLoss)计算二分类交叉熵损失使用。

In [14]:
import mindspore
import mindspore.nn as nn
import mindspore.numpy as mnp
import mindspore.ops as ops
from mindspore import Tensor

class RNN(nn.Cell):
    def __init__(self, embeddings, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, pad_idx):
        super().__init__()
        vocab_size, embedding_dim = embeddings.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim, embedding_table=Tensor(embeddings), padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout,
                           batch_first=True)
        self.fc = nn.Dense(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(1 - dropout)
        self.sigmoid = ops.Sigmoid()

    def construct(self, inputs):
        # inputs: (batch, seq_length)
        embedded = self.dropout(self.embedding(inputs))
        # embedded: (batch, seq_length, embedding_dim)
        outputs, (hidden, cell) = self.rnn(embedded)
        #outputs: (batch, seq_length, hidden_dim * num_directions)
        #hidden: (num_layers * num_directions, batch_size, hidden_dim)
        #cell: (num_layers * num_directions, batch_size, hidden_dim)
        hidden = self.dropout(mnp.concatenate((hidden[-2,:,:], hidden[-1,:,:]), axis = 1))
        output = self.fc(hidden)
        return self.sigmoid(output)

### 损失函数与优化器

完成模型主体构建后，首先根据指定的参数实例化网络；然后选择损失函数和优化器，并将其使用`nn.TrainOneStepCell`进行封装。针对本节情感分类问题的特性，即预测Positive或Negative的二分类问题，我们选择`nn.BCELoss`(二分类交叉熵损失函数)，这里也可以选择`nn.BCEWithLogitsLoss`, 其包含`sigmoid`运算，即：

```
BCEWithLogitsLoss = Sigmoid + BCELoss
```
这里使用`BECLoss`需要设置`reduction`参数为均值。而后使用`nn.WithLossCell`将其与实例化网络对象进行关联。

选择合适的损失函数后，选择`Adam`优化器，并将二者传入`TrainOneStepCell`中。

> MindSpore的设计理念为整图计算与优化，因此将Loss function和Optimizer均视为计算图的一部分，所以构造`TrainOneStepCell`作为Wrapper使用。

In [15]:
hidden_size = 256
output_size = 1
num_layers = 2
bidirectional = True
dropout = 0.5
pad_idx = vocab.tokens_to_ids('<pad>')

net = RNN(embeddings, hidden_size, output_size, num_layers, bidirectional, dropout, pad_idx)
loss = nn.BCELoss(reduction='mean')
net_with_loss = nn.WithLossCell(net, loss)
optimizer = nn.Adam(net.trainable_params())
train_one_step = nn.TrainOneStepCell(net_with_loss, optimizer)

### 训练逻辑



In [16]:
def train_one_epoch(model, train_dataset, epoch=0):
    model.set_train()
    total = train_dataset.get_dataset_size()
    loss_total = 0
    step_total = 0
    with tqdm(total=total) as t:
        t.set_description('Epoch %i' % epoch)
        for i in train_dataset.create_tuple_iterator():
            loss = model(*i)
            loss_total += loss.asnumpy()
            step_total += 1
            t.set_postfix(loss=loss_total/step_total)
            t.update(1)

### 评估指标和逻辑

In [17]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = np.around(preds)
    correct = (rounded_preds == y).astype(np.float32) #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [18]:
def evaluate(model, test_dataset, criterion, epoch=0):
    total = test_dataset.get_dataset_size()
    epoch_loss = 0
    epoch_acc = 0
    step_total = 0
    model.set_train(False)

    iterator = test_dataset.create_tuple_iterator()
    with tqdm(total=total) as t:
        t.set_description('Epoch %i' % epoch)
        for i in test_dataset.create_tuple_iterator():
            predictions = model(i[0])
            loss = criterion(predictions, i[1])
            epoch_loss += loss.asnumpy()

            acc = binary_accuracy(predictions.asnumpy(), i[1].asnumpy())
            epoch_acc += acc

            step_total += 1
            t.set_postfix(loss=epoch_loss/step_total, acc=epoch_acc/step_total)
            t.update(1)

    return epoch_loss / total

## 模型训练与保存

> MindSpore默认采用静态图模式(即Define and Run)进行训练，第一个step会进行计算图编译操作，较为耗时，但整体训练效率更高，若需要进行单步调试或使用动态图模式，可使用如下代码进行设置：
> ```python
> from mindspore import context
> context.set_context(mode=context.PYNATIVE_MODE)
> ```

In [19]:
from mindspore import save_checkpoint

num_epochs = 5
best_valid_loss = float('inf')
ckpt_file_name = os.path.join(cache_dir, 'sentiment-analysis.ckpt')

for epoch in range(num_epochs):
    train_one_epoch(train_one_step, imdb_train, epoch)
    valid_loss = evaluate(net, imdb_valid, loss, epoch)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        save_checkpoint(net, ckpt_file_name)

Epoch 4: 100%|███████████████████████████████████████████████| 117/117 [00:43<00:00,  2.71it/s, acc=0.886, loss=0.286]


## 模型加载与测试

In [20]:
from mindspore import load_checkpoint, load_param_into_net

param_dict = load_checkpoint(ckpt_file_name)
load_param_into_net(net, param_dict)

[WARNING] ME(19512:140655497611072,MainProcess):2022-02-24-03:09:19.483.329 [mindspore/common/parameter.py:338] The parameter definition is deprecated.
Please set a unique name for the parameter 'Parameter (name=Parameter, shape=(400002, 100), dtype=Float32, requires_grad=True)'.
[WARNING] ME(19512:140655497611072,MainProcess):2022-02-24-03:09:19.489.198 [mindspore/common/parameter.py:338] The parameter definition is deprecated.
Please set a unique name for the parameter 'Parameter (name=Parameter, shape=(1024, 100), dtype=Float32, requires_grad=True)'.
[WARNING] ME(19512:140655497611072,MainProcess):2022-02-24-03:09:19.490.350 [mindspore/common/parameter.py:338] The parameter definition is deprecated.
Please set a unique name for the parameter 'Parameter (name=Parameter, shape=(1024, 100), dtype=Float32, requires_grad=True)'.
[WARNING] ME(19512:140655497611072,MainProcess):2022-02-24-03:09:19.491.731 [mindspore/common/parameter.py:338] The parameter definition is deprecated.
Please se

[]

In [21]:
imdb_test = imdb_test.batch(64)
evaluate(net, imdb_test, loss)

Epoch 0: 100%|███████████████████████████████████████████████| 391/391 [00:33<00:00, 11.82it/s, acc=0.831, loss=0.389]


0.38909361658193875

## 自定义输入测试

In [22]:
score_map = {
    1: "Positive",
    0: "Negative"
}

def predict_sentiment(model, vocab, sentence):
    model.set_train(False)
    tokenized = sentence.lower().split()
    indexed = vocab.tokens_to_ids(tokenized)
    tensor = mindspore.Tensor(indexed, mindspore.int32)
    tensor = tensor.expand_dims(0)
    prediction = model(tensor)
    return score_map[int(np.round(prediction.asnumpy()))]

In [23]:
predict_sentiment(net, vocab, "This film is terrible")

'Negative'

In [24]:
predict_sentiment(net, vocab, "This film is great")

'Positive'